## Project 1 
#### Imports:

In [150]:
import numpy as np
from implementations import *
from proj1_helpers import *
from cross_validation import *
from pre_processing import *
from split_jet_num import *
from execute_code import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data set

In [349]:
""" y: class labels
    tx: features
    ids: event ids """
y, tx, ids = load_csv_data("datas/train.csv", sub_sample=False)
print("TRAIN DATA LOADED!")

no_y, tx_test, ids_test = load_csv_data("datas/test.csv", sub_sample=False)
print("TEST DATA LOADED!")

TRAIN DATA LOADED!
TEST DATA LOADED!


In [347]:
y[y == 1]

array([1])

In [355]:
(y == 1).mean(), (y == -1).mean()

(0.342668, 0.657332)

In [352]:
(y == -1).mean()

0.657332

In [353]:
y_sub, tx_sub, ids_sub = load_csv_data("datas/train.csv", sub_sample=True)
print("TRAIN DATA LOADED!")

TRAIN DATA LOADED!


In [354]:
(y_sub == 1).mean(), (y_sub == -1).mean()

(0.3286, 0.6714)

## Preprocessing 
### Replace missing values with mean, median  or normalize the dataset

In [162]:
# # Replace missing values with mean for each feature
# # train set

# means = find_mean(tx)
# tx_replaced_by_mean = replace_missing_values(tx, means)

# # test set
# means_test = find_mean(tx_test)
# tx_replaced_by_mean_test = replace_missing_values(tx_test, means_test)

In [380]:
# Replace missing values with median for each feature
# # train set
# medians = find_median(tx)
# tx_replaced_by_median = replace_missing_values(tx, medians)

# # test set
# medians_test = find_median(tx_test)
# tx_replaced_by_median_test = replace_missing_values(tx_test, medians_test)

In [381]:
# # Replace missing values with 0 and before that normalize all values without considering missing values
# std_data_tx_with_mask = standardize(clean_array(tx))
# tx_std_data_replaced_by_0 = replace_missing_values(std_data_tx_with_mask, np.full((30, 1), 0))

## Division of the dataset looking on jet num
If PRI_jet_num is zero or one then some features are -999.
Divide dataset in 4 looking on jet_num 0, 1, 2 and 3.

In [3]:
# features_jet_0, features_jet_1, features_jet_2, features_jet_3, y_jet_0, y_jet_1, y_jet_2, y_jet_3, ids_jet_0, ids_jet_1, ids_jet_2, ids_jet_3 = generate_4_sets_looking_on_jetnum(tx, y, ids)

For each set look how many missing values there are.. in order to detect how many features we want to drop!

In [4]:
# # iterate to find which columns to drop
# columns_to_remove_0 = columns_contains_just_missing_values(features_jet_0[0])
# columns_to_remove_1 = columns_contains_just_missing_values(features_jet_1[0])
# columns_to_remove_2 = columns_contains_just_missing_values(features_jet_2[0])
# columns_to_remove_3 = columns_contains_just_missing_values(features_jet_3[0])

We have noticed that we don't need to remove any features when jet num is 2 and 3. 


Infact: 


JET_NUM = 0 -> [4, 5, 6, 12, 23, 24, 25, 26, 27, 28]


JET_NUM = 1 -> [4, 5, 6, 12, 26, 27, 28]


JET_NUM = 2 -> []


JET_NUM = 3 -> []


We will drop features: 4, 5, 6, 12, 26, 27 and 28. And also we will drop feature 22 since it is the one of jet_num.

In [5]:
# # drop feature 22 since it is the one of jet_num and it will contains the same value.
# columns_to_remove_0.append(22)
# columns_to_remove_1.append(22)
# columns_to_remove_2.append(22)
# columns_to_remove_3.append(22)

Check for constant values, if I feature contains all the same values it is not important.

In [6]:
# columns_to_remove_0_b = columns_contains_same_value(features_jet_0[0])
# columns_to_remove_1_b = columns_contains_same_value(features_jet_1[0])
# columns_to_remove_2_b = columns_contains_same_value(features_jet_2[0])
# columns_to_remove_3_b = columns_contains_same_value(features_jet_3[0])

We can notice that as predicted we should removed feature 22. We have found also feature 29 to remove from column 0.
Since before we have dropped columns full of -999 (same value) we can just remove these features from each set.

JET_NUM = 0 -> [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29]


JET_NUM = 1 -> [4, 5, 6, 12, 22, 26, 27, 28]


JET_NUM = 2 -> [22]


JET_NUM = 3 -> [22]

In [7]:
# # remove columns from subset    
# features_dropped_0 = np.delete(features_jet_0[0], columns_to_remove_0_b, axis=1)
# features_dropped_1 = np.delete(features_jet_1[0], columns_to_remove_1_b, axis=1)
# features_dropped_2 = np.delete(features_jet_2[0], columns_to_remove_2_b, axis=1)
# features_dropped_3 = np.delete(features_jet_3[0], columns_to_remove_3_b, axis=1)

In [8]:
# columns_to_remove = [4, 5, 6, 12, 22, 26, 27, 28]

# tx_dropped_columns = np.delete(tx, columns_to_remove, axis=1)

## Cross validation 
K-fold cross-validation: original sample randomly partitioned into k equal sized subsamples.
Repeated k times.

In [42]:
# seed = 19
# degree = 7
# k_fold = 5

lambdas = np.logspace(-4, 0, 30) #just for ridge regression

# split data in k fold
# k_indices = build_k_indices(y, k_fold, seed)

## Test ML Methods
### Least Squares

#### Results obtained using k = 5 and replacing missing value by mean value.

In [385]:
# # store the accuracy of training data and test data
# accuracy_train = []
# accuracy_test = []
# losses = []

# for k in range(k_fold):
#     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_mean, k_indices, k, degree, least_squares)
#     accuracy_train.append(single_accuracy_train)
#     accuracy_test.append(single_accuracy_test)
#     losses.append(loss)
    
# # Just for study the behaviour
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

Iteration: 0) Accuracy train: 0.804520 - Accuracy test: 0.799740 - Loss: 0.575760

Iteration: 1) Accuracy train: 0.800310 - Accuracy test: 0.802020 - Loss: 0.585639

Iteration: 2) Accuracy train: 0.806710 - Accuracy test: 0.808080 - Loss: 0.572250

Iteration: 3) Accuracy train: 0.804575 - Accuracy test: 0.800720 - Loss: 0.576161

Iteration: 4) Accuracy train: 0.803565 - Accuracy test: 0.804200 - Loss: 0.577485

Accuracy test, mean: 0.802952, min value: 0.799740, max value: 0.808080 

Accuracy train, mean: 0.803936, min value: 0.800310, max value: 0.806710 



#### Results obtained using k = 5 and replacing missing value by median value.

In [386]:
# # store the accuracy of training data and test data
# accuracy_train = []
# accuracy_test = []
# losses = []
# weights = []

# for k in range(k_fold):
#     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_median, k_indices, k, degree, least_squares)
#     accuracy_train.append(single_accuracy_train)
#     accuracy_test.append(single_accuracy_test)
#     losses.append(loss)
#     weights.append(w)

# # Just for study the behaviour    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))


Iteration: 0) Accuracy train: 0.805525 - Accuracy test: 0.801640 - Loss: 0.573839

Iteration: 1) Accuracy train: 0.804210 - Accuracy test: 0.807480 - Loss: 0.576331

Iteration: 2) Accuracy train: 0.808065 - Accuracy test: 0.809400 - Loss: 0.569957

Iteration: 3) Accuracy train: 0.805485 - Accuracy test: 0.801400 - Loss: 0.574242

Iteration: 4) Accuracy train: 0.804755 - Accuracy test: 0.805380 - Loss: 0.575564

Accuracy test, mean: 0.805060, min value: 0.801400, max value: 0.809400 

Accuracy train, mean: 0.805608, min value: 0.804210, max value: 0.808065 



#### Results obtained using k = 5 and replacing missing value by 0 after having normalize

In [ ]:
# accuracy_train = []
# accuracy_test = []
# losses = []
# weights = []


# for k in range(k_fold):
#     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_std_data_replaced_by_0, k_indices, k, degree, least_squares)
#     accuracy_train.append(single_accuracy_train)
#     accuracy_test.append(single_accuracy_test)
#     losses.append(loss)
#     weights.append(w)

# # Just for study    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))


### Ridge regression using normal equations

#### Results obtained using k = 5 and replacing missing value by mean value.

In [ ]:
# accuracy_train = []
# accuracy_test = []
# losses = []

# for lambda_ in lambdas:
#     accuracy_train_temp = []
#     accuracy_test_temp = []
#     losses_temp = []
#     for k in range(k_fold):
#         loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_mean, k_indices, k, degree, ridge_regression, lambda_=lambda_)
#         accuracy_train_temp.append(single_accuracy_train)
#         accuracy_test_temp.append(single_accuracy_test)
#         losses_temp.append(loss)
#     accuracy_train.append(np.mean(accuracy_train_temp))
#     accuracy_test.append(np.mean(accuracy_test_temp))
#     losses.append(np.mean(losses_temp))

# # Just for study    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

#### Results obtained using k = 5 and replacing missing value by median value.

In [ ]:
# accuracy_train = []
# accuracy_test = []
# losses = []


# accuracy_train = []
# accuracy_test = []
# losses = []
# for lambda_ in lambdas:
#     accuracy_train_temp = []
#     accuracy_test_temp = []
#     losses_temp = []
#     for k in range(k_fold):
#         loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_replaced_by_median, k_indices, k, degree, ridge_regression, lambda_=lambda_)
#         accuracy_train_temp.append(single_accuracy_train)
#         accuracy_test_temp.append(single_accuracy_test)
#         losses_temp.append(loss)
#     accuracy_train.append(np.mean(accuracy_train_temp))
#     accuracy_test.append(np.mean(accuracy_test_temp))
#     losses.append(np.mean(losses_temp))
    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

## Test cross validation, division by jet_num with different ML implementations

Test jet_num_0 dataset with:
- **Least squares regression using normal equations**: <br/>
    Accuracy test, mean: 0.583375, min value: 0.511335, max value: 0.795970 <br/>
    Accuracy train, mean: 0.586020, min value: 0.511335, max value: 0.773929 
- **Ridge regression using normal equations**: <br/>
    Accuracy test, mean: 0.503224, min value: 0.476574, max value: 0.531486 <br/>
    Accuracy train, mean: 0.503237, min value: 0.473804, max value: 0.538035 
- **Linear regression using gradient descent**: <br/>
- **Linear regression using stochastic gradient descent**:
- **Logistic regression using gradient descent or SGD**:
- **Regularized logistic regression using gradient descent or SGD**:

In [10]:
# # accuracy_train = []
# # accuracy_test = []
# # losses = []
# # weights = []

# # for k in range(k_fold):
# #     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y, tx_dropped, k_indices, k, degree, least_squares)
# #     accuracy_train.append(single_accuracy_train)
# #     accuracy_test.append(single_accuracy_test)
# #     losses.append(loss)
# #     weights.append(w)

    
# # n = len(accuracy_train)
# # for i in range(n):
# #     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# # mean_accuracy_test = np.mean(accuracy_test)
# # min_accuracy_test = np.min(accuracy_test)
# # max_accuracy_test = np.max(accuracy_test)

# # mean_accuracy_train = np.mean(accuracy_train)
# # min_accuracy_train = np.min(accuracy_train)
# # max_accuracy_train = np.max(accuracy_train)

# # print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# # print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

# #LEAST SQUARE
# accuracy_train = []
# accuracy_test = []
# losses = []
# weights = []

# k_indices_0 = build_k_indices(y_jet_0, k_fold, seed)

# for k in range(k_fold):
#     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y_jet_0, features_dropped_0, k_indices_0, k, degree, least_squares)
#     accuracy_train.append(single_accuracy_train)
#     accuracy_test.append(single_accuracy_test)
#     losses.append(loss)
#     weights.append(w)

    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

Iteration: 0) Accuracy train: 0.511335 - Accuracy test: 0.513854 - Loss: 417040.188757

Iteration: 1) Accuracy train: 0.773929 - Accuracy test: 0.795970 - Loss: 0.739157

Iteration: 2) Accuracy train: 0.529597 - Accuracy test: 0.511335 - Loss: 95905.419265

Iteration: 3) Accuracy train: 0.534635 - Accuracy test: 0.549118 - Loss: 1306.744284

Iteration: 4) Accuracy train: 0.580605 - Accuracy test: 0.546599 - Loss: 21577866.290756

Accuracy test, mean: 0.583375, min value: 0.511335, max value: 0.795970 

Accuracy train, mean: 0.586020, min value: 0.511335, max value: 0.773929 



/Users/costanzavolpini/Git/project1-ML/scripts/implementations.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(a, b)[0]


In [163]:
# # RIDGE REGRESSION
# accuracy_train = []
# accuracy_test = []
# losses = []

# for lambda_ in lambdas:
#     accuracy_train_temp = []
#     accuracy_test_temp = []
#     losses_temp = []
#     for k in range(k_fold):
#         loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y_jet_0, features_dropped_0, k_indices_0, k, degree, ridge_regression, lambda_=lambda_)
#         accuracy_train_temp.append(single_accuracy_train)
#         accuracy_test_temp.append(single_accuracy_test)
#         losses_temp.append(loss)
#     accuracy_train.append(np.mean(accuracy_train_temp))
#     accuracy_test.append(np.mean(accuracy_test_temp))
#     losses.append(np.mean(losses_temp))

# # Just for study    
# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))


In [12]:
# # GRADIENT DESCEND 
# # HEREEEE! NOT WORK!
# accuracy_train = []
# accuracy_test = []
# losses = []
# weights = []
# gamma = 0.01
# max_iterations = 500

# # CHECK SHAPES!!!!!!!!!!!!!!!!!!!!! and correct!
# k_indices_0 = build_k_indices(y_jet_0, k_fold, seed)

# initial_w = np.zeros(features_dropped_0.shape[1])

# for k in range(k_fold):
#     loss, w, single_accuracy_train, single_accuracy_test = cross_validation(y_jet_0, features_dropped_0, k_indices_0, k, degree, gradient_descent, initial_w=initial_w, max_iters=max_iterations, gamma=gamma)
#     accuracy_train.append(single_accuracy_train)
#     accuracy_test.append(single_accuracy_test)
#     losses.append(loss)
#     weights.append(w)

# n = len(accuracy_train)
# for i in range(n):
#     print("Iteration: %d) Accuracy train: %f - Accuracy test: %f - Loss: %f\n" % (i, accuracy_train[i], accuracy_test[i], losses[i]))

# mean_accuracy_test = np.mean(accuracy_test)
# min_accuracy_test = np.min(accuracy_test)
# max_accuracy_test = np.max(accuracy_test)

# mean_accuracy_train = np.mean(accuracy_train)
# min_accuracy_train = np.min(accuracy_train)
# max_accuracy_train = np.max(accuracy_train)

# print("Accuracy test, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_test, min_accuracy_test, max_accuracy_test))
# print("Accuracy train, mean: %f, min value: %f, max value: %f \n" %(mean_accuracy_train, min_accuracy_train, max_accuracy_train))

/Users/costanzavolpini/Git/project1-ML/scripts/helpers.py:5: RuntimeWarning: overflow encountered in square
  return 1/(len(y)) * np.sum(e**2)
/Users/costanzavolpini/Git/project1-ML/scripts/proj1_helpers.py:30: RuntimeWarning: invalid value encountered in less_equal
  y_pred[np.where(y_pred <= 0)] = -1
/Users/costanzavolpini/Git/project1-ML/scripts/proj1_helpers.py:31: RuntimeWarning: invalid value encountered in greater
  y_pred[np.where(y_pred > 0)] = 1


Iteration: 0) Accuracy train: 0.000000 - Accuracy test: 0.000000 - Loss: nan

Iteration: 1) Accuracy train: 0.000000 - Accuracy test: 0.000000 - Loss: nan

Iteration: 2) Accuracy train: 0.000000 - Accuracy test: 0.000000 - Loss: nan

Iteration: 3) Accuracy train: 0.000000 - Accuracy test: 0.000000 - Loss: nan

Iteration: 4) Accuracy train: 0.000000 - Accuracy test: 0.000000 - Loss: nan

Accuracy test, mean: 0.000000, min value: 0.000000, max value: 0.000000 

Accuracy train, mean: 0.000000, min value: 0.000000, max value: 0.000000 



### Generate Prediction (example)

In [34]:
# y_test_predicted = []
# test_poly = build_poly(tx_median_test, degree)
# y_test_predicted = predict_labels(weights[0], test_poly)
# create_csv_submission(ids_test, y_test_predicted, "submission-7")

array([[ 0.28075458,  0.03965436,  0.40651625, ...,  0.61755292,
        -1.30266251,  0.39818214],
       [ 1.63061547,  0.63344061,  1.05925506, ...,  0.        ,
         0.        , -0.23179611],
       [-0.51771811, -0.64151886, -0.37784339, ...,  0.        ,
         0.        , -0.10829837],
       ...,
       [ 0.34569571, -1.22366112,  0.43998942, ...,  0.        ,
         0.        ,  0.23416077],
       [-0.72942585, -0.11217409, -0.50441304, ...,  0.        ,
         0.        , -0.41106251],
       [ 0.13958259, -1.31829705, -0.3187276 , ...,  0.        ,
         0.        , -0.22539659]])

In [58]:
# build_k_indices(y_0, 5, 19).shape, y_0.shape

((5, 397), (1985,))

In [356]:
# Test my new file!!!!!!!!!!! DA QUI:
y_test = np.zeros(tx_test.shape[0])
# tx_med = replace_set_median(tx)
tx_out = tx
# tx_out = outlier_removal(tx_med, 95, 1, per_top = True, per_bot = True)
tx_0, tx_1, tx_2, tx_3, y_0, y_1, y_2, y_3, ids_0, ids_1, ids_2, ids_3 = divide_dataset_looking_jetnum_and_remove_features(y, tx_out, ids)

In [357]:
tx_0 = tx_0.copy()
tx_1 = tx_1.copy()
tx_2 = tx_2.copy()
tx_3 = tx_3.copy()

In [340]:
acc0, m0, w0 = execute_all_methods(y_0, tx_0, ids_0, True, 8, lambda_=0.0001, initial_w=None, max_iters=1000, gamma=1e-3)
print("Selected method ", m0)
print("with accuracy %f" %(acc0))

/Users/costanzavolpini/Git/project1-ML/scripts/implementations.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(a, b)[0]


KeyboardInterrupt: 

In [343]:
acc0, w0 = execute_one_method(y_0, tx_0, ids_0, "logistic", True, 10, logistic_regression, initial_w=None, max_iters=1000, gamma=1e-3)
print("Selected method ", "Logistic")
print("with accuracy %f" %(acc0))

Selected method  Logistic
with accuracy 0.728636


In [ ]:
#QUESTO:

In [359]:
acc0, w0 = execute_one_method(y_0, tx_0, ids_0, "logistic", True, 8, logistic_regression, initial_w=None, max_iters=1000, gamma=1e-3)
print("Selected method ", "Logistic")
print("with accuracy %f" %(acc0))

Selected method  Logistic
with accuracy 0.708568


In [325]:
acc0, w0 = execute_one_method(y_0, tx_0, ids_0, "ridge", True, 7, ridge_regression, lambda_=0.01)
print("Selected method ", "Ridge")
print("with accuracy %f" %(acc0))

Selected method  Ridge
with accuracy 0.760134


In [339]:
acc0, w0 = execute_one_method(y_0, tx_0, ids_0, "ridge", True, 6, ridge_regression, lambda_=0.1)
print("Selected method ", "Ridge")
print("with accuracy %f" %(acc0))

Selected method  Ridge
with accuracy 0.746071


with accuracy 0.684826


In [251]:
acc1, m1, w1 = execute_all_methods(y_1, tx_1, ids_1, True, 6, lambda_=0.0001, initial_w=None, max_iters=800, gamma=1e-3)
print("Selected method ", m1)
print("with accuracy %f" %(acc1))

/Users/costanzavolpini/Git/project1-ML/scripts/implementations.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(a, b)[0]


Selected method  LOGISTIC REGRESSION
with accuracy 0.744156


In [270]:
acc2, m2, w2 = execute_all_methods(y_2, tx_2, ids_2, True, 5, lambda_=0.0001, initial_w=None, max_iters=3000, gamma=1e-3)
print("Selected method ", m2)
print("with accuracy %f" %(acc2))

/Users/costanzavolpini/Git/project1-ML/scripts/implementations.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(a, b)[0]


Selected method  LOGISTIC REGRESSION
with accuracy 0.767308


In [297]:
acc3, m3, w3 = execute_all_methods(y_3, tx_3, ids_3, True, 10, lambda_=0.0001, initial_w=None, max_iters=800, gamma=1e-2)
print("Selected method ", m3)
print("with accuracy %f" %(acc3))

/Users/costanzavolpini/Git/project1-ML/scripts/implementations.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(a, b)[0]


Selected method  LOGISTIC REGRESSION
with accuracy 0.709302


In [180]:
tx_0_t, tx_1_t, tx_2_t, tx_3_t, _, _, _, _, ids_0_t, ids_1_t, ids_2_t, ids_3_t = divide_dataset_looking_jetnum_and_remove_features(y_test, tx_test, ids_test)


In [181]:
generate_submission(tx_0_t, tx_1_t, tx_2_t, tx_3_t, ids_0_t, ids_1_t, ids_2_t, ids_3_t, w0, w1, w2, w3, "submission-28-10-to-submit", 7)


[[ 3.50000e+05 -1.00000e+00]
 [ 3.50002e+05  1.00000e+00]
 [ 3.50003e+05  1.00000e+00]
 ...
 [ 9.18232e+05 -1.00000e+00]
 [ 9.18235e+05  1.00000e+00]
 [ 9.18237e+05 -1.00000e+00]]


array([-1., -1.,  1., ...,  1.,  1., -1.])

In [107]:
len(w1), len(w2), len(w3), len(w0)

(155, 204, 204, 127)

In [182]:
w_final = np.concatenate((w0, w1, w2, w3))
ids_final = np.concatenate((ids_0, ids_1, ids_2, ids_3))

In [183]:
preddd = generate_submission(tx_0, tx_1, tx_2, tx_3, ids_0, ids_1, ids_2, ids_3, w0, w1, w2, w3, "submission-28-10-to-submit", 7)


[[ 1.0030e+05 -1.0000e+00]
 [ 1.0035e+05 -1.0000e+00]
 [ 1.0050e+05 -1.0000e+00]
 ...
 [ 3.4945e+05 -1.0000e+00]
 [ 3.4955e+05  1.0000e+00]
 [ 3.4970e+05 -1.0000e+00]]


In [184]:
np.sum(preddd == y)/len(y)

0.8058

In [187]:
    y_test_predicted0 = []
    test_poly0 = build_poly(tx_0, degree)
    test_poly0 = replace_set_normalize(test_poly0)
    y_test_predicted0 = predict_labels(w0, test_poly0)
np.sum(y_test_predicted0 == y_0)/len(y_0)


ValueError: shapes (1985,37) and (127,) not aligned: 37 (dim 1) != 127 (dim 0)

In [112]:
    y_test_predicted1 = []
    test_poly1 = build_poly(tx_1, degree)
    test_poly1 = replace_set_normalize(test_poly1)
    y_test_predicted1 = predict_labels(w1, test_poly1)
np.sum(y_test_predicted1 == y_1)/len(y_1)

0.7766233766233767

In [113]:
    y_test_predicted2 = []
    test_poly2 = build_poly(tx_2, degree)
    test_poly2 = replace_set_normalize(test_poly2)
    y_test_predicted2 = predict_labels(w2, test_poly2)
np.sum(y_test_predicted2 == y_2)/len(y_2)

0.8293384467881112

In [114]:
    y_test_predicted3 = []
    test_poly3 = build_poly(tx_3, degree)
    test_poly3 = replace_set_normalize(test_poly3)
    y_test_predicted3 = predict_labels(w3, test_poly3)
np.sum(y_test_predicted3 == y_3)/len(y_3)

0.7893518518518519

In [301]:
# 1. Load datasets
y, tx, ids = load_csv_data("datas/train.csv", sub_sample=False)
no_y, tx_test, ids_test = load_csv_data("datas/test.csv", sub_sample=False)

# 2. Divide the train dataset looking on jet_num feature (column 22 of tx)
tx_0, tx_1, tx_2, tx_3, y_0, y_1, y_2, y_3, ids_0, ids_1, ids_2, ids_3 = divide_dataset_looking_jetnum_and_remove_features(y, tx, ids)

# 3. Execute all methods on all subsets
acc0, w0 = execute_one_method(y_0, tx_0, ids_0, "logistic", True, 2, logistic_regression, initial_w=None, max_iters=1000, gamma=1e-3)
# print("Selected method ", m0)
print("with accuracy %f" %(acc0))

acc1, w1 = execute_one_method(y_1, tx_1, ids_1, "logistic", True, 2, logistic_regression, initial_w=None, max_iters=1000, gamma=1e-3)
# print("Selected method ", m1)
print("with accuracy %f" %(acc1))

acc2, w2 = execute_one_method(y_2, tx_2, ids_2, "logistic", True, 2, logistic_regression, initial_w=None, max_iters=1000, gamma=1e-3)
# print("Selected method ", m2)
print("with accuracy %f" %(acc2))

acc3, w3 = execute_one_method(y_3, tx_3, ids_3, "logistic", True, 2, logistic_regression, initial_w=None, max_iters=1000, gamma=1e-3)
# print("Selected method ", m3)
print("with accuracy %f" %(acc3))

# 4. Divide the test dataset looking on jet_num feature (column 22 of tx)
tx_0_t, tx_1_t, tx_2_t, tx_3_t, _, _, _, _, ids_0_t, ids_1_t, ids_2_t, ids_3_t = divide_dataset_looking_jetnum_and_remove_features(no_y, tx_test, ids_test)

# 5. Generate submission
final_prediction = generate_submission(tx_0, tx_1, tx_2, tx_3, ids_0, ids_1, ids_2, ids_3, w0, w1, w2, w3, "submission-28-10-to-submit", [8, 6, 2, 10])

print(np.sum(final_prediction == y)/len(y))

with accuracy 0.684826
with accuracy 0.675522
with accuracy 0.735762
with accuracy 0.586282


ValueError: shapes (99913,145) and (37,) not aligned: 145 (dim 1) != 37 (dim 0)

In [10]:
# 0. Imports
import numpy as np
from implementations import *
from proj1_helpers import *
from cross_validation import *
from pre_processing import *
from split_jet_num import *
from execute_code import *

In [3]:

# 1. Load datasets
y, tx, ids = load_csv_data("datas/train.csv", sub_sample=False)
no_y, tx_test, ids_test = load_csv_data("datas/test.csv", sub_sample=False)

In [11]:
# 2. Divide the train dataset looking on jet_num feature (column 22 of tx)
tx_0, tx_1, tx_2, tx_3, y_0, y_1, y_2, y_3, ids_0, ids_1, ids_2, ids_3 = divide_dataset_looking_jetnum_and_remove_features(y, tx, ids)


# 3. Execute all methods on all subsets
acc0, w0 = execute_one_method(y_0, tx_0, ids_0, "logistic", True, 8, logistic_regression, lambda_=0.0001, initial_w=None, max_iters=1000, gamma=1e-3)
print("Selected method ", "Ridge")
print("with accuracy %f" %(acc0))

NameError: name 'build_k_indices' is not defined